In [39]:
# ============================================================================
# COMPLETE QUANTUM RESERVOIR COMPUTING IMPLEMENTATION
# Week 1: Baseline implementation with Quantum + Classical Reservoirs
# ============================================================================

import cudaq
import numpy as np
from itertools import combinations
from typing import List
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
from copy import deepcopy

cudaq.set_target("qpp-cpu")  # Set target once at the beginning

@cudaq.kernel
def qrc_circuit_cudaq(num_qubits: int, params: List[float]):
    qubits = cudaq.qvector(num_qubits)
    param_idx = 0
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            x.ctrl(qubits[i], qubits[j])
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            x.ctrl(qubits[i], qubits[j])
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            x.ctrl(qubits[i], qubits[j])
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1

class OCQRC_CUDAQ:
    """
    Onion Classical-Quantum Reservoir Computing with CUDA-Q
    Based on arXiv:2505.22837
    """
    
    def __init__(self, num_qubits=6, crc_size=850, 
                 n_reservoirs=3, f_bs=[0.11, 0.1375, 0.12375], b=-0.33,
                 ridge_alpha=3e-4,
                 warmup_steps=5,
                 use_classical=True, use_quantum=True,
                 target="qpp-cpu"):
        """
        Args:
            num_qubits: Number of qubits per quantum reservoir
            crc_size: Size of classical reservoir
            n_reservoirs: Number of independent quantum reservoirs
            f_bs: Feedback parameters for each reservoir
            b: Reservoir parameter
            ridge_alpha: Ridge regression regularization
            use_classical: Enable classical reservoir
            use_quantum: Enable quantum reservoir(s)
            target: CUDA-Q target (qpp-cpu, nvidia, tensornet, etc.)
        """
        self.num_qubits = num_qubits
        self.crc_size = crc_size
        self.n_reservoirs = n_reservoirs
        self.f_bs = f_bs
        self.b = b
        self.ridge_alpha = ridge_alpha
        self.use_classical = use_classical
        self.use_quantum = use_quantum
        self.warmup_steps = warmup_steps
        
        # Set target
        cudaq.set_target(target)
        
        # Initialize quantum reservoir
        if self.use_quantum:
            self._init_quantum_reservoir()
        
        # Initialize classical reservoir
        if self.use_classical:
            self._init_classical_reservoir()
        
        # Ridge regression model
        self.ridge = Ridge(alpha=self.ridge_alpha)
        
    def _init_quantum_reservoir(self):
        """Initialize quantum reservoir observables and states"""
        # Generate observable indices for efficient calculation
        self._int_observables = []
        
        # Single qubit Z_i observables
        for i in range(self.num_qubits):
            obs_int = 1 << i  # Binary representation
            self._int_observables.append(obs_int)
        
        # Two-qubit Z_i Z_j observables
        for i, j in combinations(range(self.num_qubits), 2):
            obs_int = (1 << i) | (1 << j)
            self._int_observables.append(obs_int)
        
        self._int_observables = np.array(self._int_observables, dtype=int)
        self.n_observables = len(self._int_observables)
        
        # Initialize reservoir states (Z expectation values) - START AT ZERO!
        self.last_outputs = [np.zeros(self.num_qubits) 
                            for _ in range(self.n_reservoirs)]
        self.init_qrc_states = deepcopy(self.last_outputs)
        
    def _init_classical_reservoir(self):
        """Initialize classical Echo State Network"""
        np.random.seed(2)  # For reproducibility
        
        self.inSize = 1
        self.resSize = self.crc_size
        
        # Input weights
        self.Win = (np.random.rand(self.resSize, 1 + self.inSize) - 0.5) * 1
        
        # Reservoir weights
        self.W = np.random.rand(self.resSize, self.resSize) - 0.5
        
        # Normalize to spectral radius = 1.0
        eigenvalues = np.linalg.eigvals(self.W)
        rhoW = max(abs(eigenvalues))
        self.W *= 1.00 / rhoW
        
        # Initial state
        self.init_crc_state = np.tanh(np.ones((self.resSize, 1)) / np.sqrt(self.resSize))
        self.x = self.init_crc_state.copy()
        
    def calc_observables(self, state_vector):
        """Calculate all observables from statevector efficiently"""
        # Get probabilities
        probs = (np.conj(state_vector) * state_vector).real
        
        # Calculate expectation values using bitwise operations
        exps = np.zeros(self.n_observables, dtype=float)
        for i, prob in enumerate(probs):
            for o, obs_int in enumerate(self._int_observables):
                # Count number of 1s in (obs_int & i) - determines sign
                sign = (-1) ** bin(obs_int & i).count("1")
                exps[o] += sign * prob
        
        return exps
    
    def evolve_qrc(self, input_value):
        """Evolve all quantum reservoirs with CORRECT parameter encoding"""
        if not self.use_quantum:
            return np.array([])
        
        all_observables = []
        
        for res_idx in range(self.n_reservoirs):
            # Get previous Z values
            zs = self.last_outputs[res_idx]
            
            # Clip to valid arccos domain
            zs = np.clip(zs, -1, 1)
            
            # Compute feedback and encoded parameters
            f_b = input_value * self.b * self.f_bs[res_idx]
            arccos_zs = np.arccos(zs) * self.b
            
            # CRITICAL FIX: Expand vars = [f_b] + zs[:-1] + zs*4
            # This matches Qiskit's parameter structure exactly
            params_list = [f_b]  # Position 0
            params_list.extend(arccos_zs[:-1])  # Positions 1 to num_qubits-1 (missing last z!)
            params_list.extend(list(arccos_zs) * 4)  # Repeat all zs 4 times
            
            # Get statevector
            state = cudaq.get_state(qrc_circuit_cudaq, self.num_qubits, params_list)
            state_array = np.array(state)
            
            # Calculate observables
            observables = self.calc_observables(state_array)
            
            # Update state (first num_qubits observables are Z_i)
            self.last_outputs[res_idx] = observables[:self.num_qubits]
            
            all_observables.extend(observables)
        
        return np.array(all_observables)
    
    def evolve_crc(self, input_value):
        """Evolve classical reservoir"""
        if not self.use_classical:
            return np.array([])
        
        # Normalize input
        u = input_value / 100.0
        
        # ESN evolution: x = tanh(Win·[1, u] + W·x)
        self.x = np.tanh(
            np.dot(self.Win, np.vstack((1, u))) + 
            np.dot(self.W, self.x)
        )
        
        # Return features: [1, u, x]
        features = np.vstack((1, u, self.x)).flatten()
        return features
    
    def combined(self, input_value):
        """Evolve both quantum and classical reservoirs"""
        qrc_features = self.evolve_qrc(input_value)
        crc_features = self.evolve_crc(input_value)
        return np.concatenate([qrc_features, crc_features])
    
    def reset_reservoirs(self):
        """Reset all reservoirs to initial states"""
        if self.use_quantum:
            self.last_outputs = deepcopy(self.init_qrc_states)
        if self.use_classical:
            self.x = self.init_crc_state.copy()
    
    def train(self, train_data):
        """
        Train on time series data
        Args:
            train_data: numpy array of shape (n_series, n_timesteps)
        """
        all_states = []
        all_targets = []
        
        n_series = train_data.shape[0]
        
        for series_idx in range(n_series):
            series = train_data[series_idx]
            
            # Reset reservoirs
            self.reset_reservoirs()
            
            # Warmup: discard initial timesteps for better stabilization
            for _ in range(self.warmup_steps):
                _ = self.combined(series[0])
            
            # Collect training data
            for t in range(len(series) - 1):
                state = self.combined(series[t])
                target = series[t + 1]
                
                # Check for NaN
                if not np.all(np.isfinite(state)):
                    print(f"Warning: NaN in state at series {series_idx}, time {t}")
                    state = np.nan_to_num(state, nan=0.0)
                
                all_states.append(state)
                all_targets.append(target)
        
        # Convert to arrays
        X = np.array(all_states)
        y = np.array(all_targets)
        
        # Check for NaN before fitting
        if not np.all(np.isfinite(X)):
            raise ValueError("NaN/Inf in training features!")
        if not np.all(np.isfinite(y)):
            raise ValueError("NaN/Inf in training targets!")
        
        # Fit ridge regression
        self.ridge.fit(X, y)
        
        print(f"Training complete: {len(all_states)} samples, {X.shape[1]} features")
    
    def predict(self, test_data, n_predict=20):
        """
        Predict future timesteps
        Args:
            test_data: Initial sequence, shape (n_test, n_initial_steps)
            n_predict: Number of steps to predict
        Returns:
            predictions: shape (n_test, n_predict)
        """
        n_test = test_data.shape[0]
        predictions = np.zeros((n_test, n_predict))
        
        for test_idx in range(n_test):
            series = test_data[test_idx]
            
            # Reset reservoirs
            self.reset_reservoirs()
            
            # Warmup for better stabilization
            for _ in range(self.warmup_steps):
                _ = self.combined(series[0])
            
            # Evolve through initial data
            for t in range(len(series) - 1):
                _ = self.combined(series[t])
            
            # Last known value
            last_value = series[-1]
            
            # Iteratively predict
            for t in range(n_predict):
                state = self.combined(last_value)
                
                # NaN protection
                if not np.all(np.isfinite(state)):
                    print(f"Warning: NaN in state at test {test_idx}, pred step {t}")
                    state = np.nan_to_num(state, nan=0.0)
                
                # Predict next value
                pred = self.ridge.predict(state.reshape(1, -1))[0]
                predictions[test_idx, t] = pred
                last_value = pred
        
        return predictions


def generate_damped_oscillator_data(n_train=10, n_test=4, n_points=26, seed=1):
    """
    Generate damped harmonic oscillator data
    Formula: 30*exp(-0.05*t)*(cos(ω*t) + cos(ω*t/√2)) + noise
    """
    rng = np.random.Generator(np.random.PCG64(seed))
    
    t = np.linspace(0, 26, num=n_points)
    n_total = n_train + n_test
    
    # Random frequencies
    oscis = 0.5 + rng.standard_normal(n_total) * 0.4
    
    # Generate data
    data = np.zeros((n_total, n_points))
    for j in range(n_total):
        omega = oscis[j]
        data[j, :] = (
            np.exp(-0.05 * t) * 
            (np.cos(omega * t) + np.cos(omega * t / np.sqrt(2))) * 30 +
            rng.standard_normal(n_points) * 0.3
        )
    
    return data


print("✅ OCQRC_CUDAQ class defined successfully!")
print("✅ Test data generation function ready!")
print("\nNext: Run the testing cell below to evaluate performance.")

✅ OCQRC_CUDAQ class defined successfully!
✅ Test data generation function ready!

Next: Run the testing cell below to evaluate performance.


In [40]:
# ============================================================================
# Diagnostics and helper utilities
# ============================================================================

def compute_forecast_metrics(ground_truth, predictions):
    flat_truth = ground_truth.flatten()
    flat_pred = predictions.flatten()
    mse = mean_squared_error(flat_truth, flat_pred)
    rmse = np.sqrt(mse)
    std = np.std(flat_truth)
    nrmse = rmse / (std + 1e-12)
    r2 = r2_score(flat_truth, flat_pred)
    return {"mse": mse, "rmse": rmse, "nrmse": nrmse, "r2": r2}

def summarize_quantum_dynamics(model, series, n_steps=20):
    model.reset_reservoirs()
    records = []
    for _ in range(model.warmup_steps):
        _ = model.evolve_qrc(series[0])
    for t in range(min(len(series), n_steps)):
        obs = model.evolve_qrc(series[t])
        if obs.size:
            records.append(obs.copy())
    if not records:
        return {"feature_mean": 0.0, "feature_std": 0.0, "feature_var": 0.0}
    arr = np.stack(records)
    obs_std = float(np.std(arr, axis=0).mean())
    obs_var = float(np.var(arr, axis=0).mean())
    obs_mean = float(np.mean(arr))
    return {"feature_mean": obs_mean, "feature_std": obs_std, "feature_var": obs_var}

def display_prediction_sample(results_dict, config_name, sample_idx=0, max_steps=None):
    if config_name not in results_dict:
        print(f"No configuration named {config_name!r} in results.")
        return
    entry = results_dict[config_name]
    truth = entry["ground_truth"][sample_idx]
    pred = entry["predictions"][sample_idx]
    steps = len(truth) if max_steps is None else min(max_steps, len(truth))
    print(f"Configuration: {config_name}")
    print(f"Sample index: {sample_idx}")
    print("step  expected      predicted     error")
    for t in range(steps):
        expected = truth[t]
        forecast = pred[t]
        print(f"{t:>4}  {expected: .6f}  {forecast: .6f}  {forecast - expected: .6f}")
    abs_err = float(np.abs(truth[:steps] - pred[:steps]).mean())
    print(f"Mean absolute error over first {steps} steps: {abs_err:.6f}")

def run_qubit_scaling_diagnostic(qubit_list, model_kwargs=None, data_kwargs=None, n_predict=20, init_points=6):
    model_kwargs = model_kwargs or {}
    data_defaults = {"n_train": 10, "n_test": 4, "n_points": 26, "seed": 1}
    if data_kwargs:
        data_defaults.update(data_kwargs)
    dataset = generate_damped_oscillator_data(**data_defaults)
    train_data = dataset[:data_defaults["n_train"]]
    test_data = dataset[data_defaults["n_train"]:]
    print("=" * 70)
    print("QUBIT SCALING DIAGNOSTIC")
    print("=" * 70)
    results = []
    for n in qubit_list:
        print(f"\n[Qubits: {n}]")
        model = OCQRC_CUDAQ(num_qubits=n, use_classical=False, use_quantum=True, **model_kwargs)
        model.train(train_data)
        preds = model.predict(test_data[:, :init_points], n_predict=n_predict)
        metrics = compute_forecast_metrics(test_data[:, init_points:], preds)
        stats = summarize_quantum_dynamics(model, train_data[0])
        dim = 2 ** n
        features = model.n_observables * model.n_reservoirs if model.use_quantum else 0
        print(f"  Hilbert dim: {dim}")
        print(f"  Features: {features}")
        print(f"  MSE: {metrics['mse']:.2f}, RMSE: {metrics['rmse']:.2f}, NRMSE: {metrics['nrmse']:.4f}, R²: {metrics['r2']:.4f}")
        print(f"  Feature mean: {stats['feature_mean']:.4f}, std: {stats['feature_std']:.4f}")
        results.append({"qubits": n, "metrics": metrics, "stats": stats})
    print("Diagnostics complete.")
    return results


## Baseline evaluation
Run after defining the utilities to compare reservoir configurations.


In [41]:
# ============================================================================
# WEEK 1 TESTING: Evaluate OCQRC Performance
# Goal: Achieve positive R² correlation (target: R² > 0.5)
# ============================================================================

print("="*70)
print("WEEK 1: OCQRC BASELINE EVALUATION")
print("="*70)

# Generate test data
print("\n[1/4] Generating damped harmonic oscillator data...")
data = generate_damped_oscillator_data(n_train=10, n_test=4, n_points=26, seed=1)
train_data = data[:10]
test_data = data[10:]

print(f"  Train: {train_data.shape}")
print(f"  Test: {test_data.shape}")

# Test configurations
configs = [
    ("Classical Only", True, False),
    ("Quantum Only", False, True),
    ("Quantum+Classical", True, True)
]

results = {}

for name, use_classical, use_quantum in configs:
    print(f"\n[2/4] Testing: {name}")
    print("-" * 70)
    
    # Create model
    model = OCQRC_CUDAQ(
        num_qubits=6,
        crc_size=850,
        n_reservoirs=3,
        f_bs=[0.11, 0.1375, 0.12375],
        b=-0.33,
        ridge_alpha=3e-4,
        use_classical=use_classical,
        use_quantum=use_quantum,
        target="qpp-cpu"
    )
    
    # Train
    print(f"  [Training on {train_data.shape[0]} series...]")
    model.train(train_data)
    
    # Predict
    print(f"  [Predicting 20 steps from first 6 points...]")
    predictions = model.predict(test_data[:, :6], n_predict=20)
    
    # Evaluate
    ground_truth = test_data[:, 6:]
    flat_truth = ground_truth.flatten()
    flat_pred = predictions.flatten()
    r2 = r2_score(flat_truth, flat_pred)
    mse = mean_squared_error(flat_truth, flat_pred)
    rmse = np.sqrt(mse)
    std = np.std(flat_truth)
    nrmse = rmse / (std + 1e-12)
    
    results[name] = {
        'r2': r2,
        'mse': mse,
        'predictions': predictions,
        'ground_truth': ground_truth,
        'rmse': rmse,
        'nrmse': nrmse
    }
    
    print(f"  MSE: {mse:.2f}, RMSE: {rmse:.2f}, NRMSE: {nrmse:.4f}, R²: {r2:.4f}")

# Summary
print("\n" + "="*70)
print("[3/4] RESULTS SUMMARY")
print("="*70)
for name, res in results.items():
    status = "✅ PASS" if res['r2'] > 0.0 else "❌ FAIL"
    print(f"  {name:20s}: MSE = {res['mse']:7.2f}, NRMSE = {res['nrmse']:7.4f}, R² = {res['r2']:7.4f}  {status}")

# Check Week 1 goal
print("\n" + "="*70)
print("[4/4] WEEK 1 GOAL CHECK")
print("="*70)
best_r2 = max(res['r2'] for res in results.values())
if best_r2 > 0.5:
    print(f"  🎉 EXCELLENT! Best R² = {best_r2:.4f} > 0.5")
    print("  ✅ Week 1 goal achieved! Ready for Week 2.")
elif best_r2 > 0.0:
    print(f"  ✅ GOOD! Best R² = {best_r2:.4f} (positive correlation)")
    print("  💡 Consider: increase n_reservoirs, tune hyperparameters")
else:
    print(f"  ⚠️  Best R² = {best_r2:.4f} (needs improvement)")
    print("  💡 Debug: check for NaN, verify data generation")



WEEK 1: OCQRC BASELINE EVALUATION

[1/4] Generating damped harmonic oscillator data...
  Train: (10, 26)
  Test: (4, 26)

[2/4] Testing: Classical Only
----------------------------------------------------------------------
  [Training on 10 series...]
Training complete: 250 samples, 852 features
  [Predicting 20 steps from first 6 points...]
  MSE: 185.63, RMSE: 13.62, NRMSE: 0.9882, R²: 0.0234

[2/4] Testing: Quantum Only
----------------------------------------------------------------------
  [Training on 10 series...]
Training complete: 250 samples, 63 features
  [Predicting 20 steps from first 6 points...]
  MSE: 278.44, RMSE: 16.69, NRMSE: 1.2103, R²: -0.4649

[2/4] Testing: Quantum+Classical
----------------------------------------------------------------------
  [Training on 10 series...]
Training complete: 250 samples, 915 features
  [Predicting 20 steps from first 6 points...]
  MSE: 70.80, RMSE: 8.41, NRMSE: 0.6103, R²: 0.6275

[3/4] RESULTS SUMMARY
  Classical Only      : MS

In [42]:
# Inspect predictions vs ground truth for a chosen configuration
display_prediction_sample(results, config_name='Quantum Only', sample_idx=0)


Configuration: Quantum Only
Sample index: 0
step  expected      predicted     error
   0  -35.578434  -38.600442  -3.022008
   1  -35.761099  -38.780573  -3.019474
   2  -28.446608  -24.990067   3.456541
   3  -16.700774  -9.418591   7.282183
   4  -4.642688   0.835724   5.478412
   5   6.280768   8.017192   1.736424
   6   12.866558   12.147806  -0.718752
   7   14.799790   6.280987  -8.518803
   8   12.937450  -0.495137  -13.432588
   9   9.251257  -7.621615  -16.872871
  10   4.377753  -11.805846  -16.183599
  11   1.127388  -12.429258  -13.556646
  12  -1.189487  -10.039041  -8.849554
  13  -1.179924  -5.726085  -4.546161
  14  -0.221376  -0.128794   0.092583
  15   1.374224   6.438539   5.064315
  16   2.064434   12.462812   10.398378
  17   2.291690   16.453966   14.162276
  18   1.046105   16.125179   15.079074
  19  -2.169868   8.280001   10.449869
Mean absolute error over first 20 steps: 8.096026


## Quantum-only scaling diagnostic
Use this to gauge feature richness across qubit counts.


In [43]:
# ============================================================================
# Diagnostic: assess quantum reservoir scaling
# ============================================================================

qubit_sweep = [4, 6, 8]
diagnostic_results = run_qubit_scaling_diagnostic(
    qubit_sweep,
    model_kwargs={
        "n_reservoirs": 3,
        "f_bs": [0.11, 0.1375, 0.12375],
        "b": -0.33,
        "ridge_alpha": 3e-4,
        "target": "qpp-cpu"
    },
    data_kwargs={
        "seed": 1
    },
    n_predict=20,
    init_points=6
)


QUBIT SCALING DIAGNOSTIC

[Qubits: 4]
Training complete: 250 samples, 30 features
  Hilbert dim: 16
  Features: 30
  MSE: 155.82, RMSE: 12.48, NRMSE: 0.9054, R²: 0.1802
  Feature mean: 0.0318, std: 0.2727

[Qubits: 6]
Training complete: 250 samples, 63 features
  Hilbert dim: 64
  Features: 63
  MSE: 278.44, RMSE: 16.69, NRMSE: 1.2103, R²: -0.4649
  Feature mean: -0.0030, std: 0.1449

[Qubits: 8]
Training complete: 250 samples, 108 features
  Hilbert dim: 256
  Features: 108
  MSE: 190.21, RMSE: 13.79, NRMSE: 1.0004, R²: -0.0007
  Feature mean: -0.0191, std: 0.0936
Diagnostics complete.


## Hyperparameter sweep
Loop over quantum-only settings to probe sensitivity.


In [44]:
# Hyperparameter sweep for quantum-only reservoir
if 'train_data' not in globals() or 'test_data' not in globals():
    data = generate_damped_oscillator_data(n_train=10, n_test=4, n_points=26, seed=1)
    train_data = data[:10]
    test_data = data[10:]
b_values = [-0.50, -0.40, -0.33]
f_scale_sets = [
    [0.08, 0.10, 0.12],
    [0.11, 0.1375, 0.12375],
    [0.14, 0.16, 0.18],
    [0.09, 0.11, 0.13, 0.15]
]
warmup_options = [5, 8]
reservoir_options = [3, 4]
qubit_options = [4, 6]
sweep_results = []
for num_qubits in qubit_options:
    for b in b_values:
        for f_bs in f_scale_sets:
            for n_reservoirs in reservoir_options:
                if n_reservoirs != len(f_bs):
                    continue
                for warmup in warmup_options:
                    model = OCQRC_CUDAQ(num_qubits=num_qubits,
                                        n_reservoirs=n_reservoirs,
                                        f_bs=f_bs,
                                        b=b,
                                        warmup_steps=warmup,
                                        ridge_alpha=3e-4,
                                        use_classical=False,
                                        use_quantum=True,
                                        target="qpp-cpu")
                    model.train(train_data)
                    preds = model.predict(test_data[:, :6], n_predict=20)
                    metrics = compute_forecast_metrics(test_data[:, 6:], preds)
                    stats = summarize_quantum_dynamics(model, train_data[0])
                    sweep_results.append({
                        "num_qubits": num_qubits,
                        "b": b,
                        "f_bs": f_bs,
                        "warmup": warmup,
                        "n_reservoirs": n_reservoirs,
                        "metrics": metrics,
                        "stats": stats
                    })
                    print(f"Q={num_qubits} b={b:.2f} warmup={warmup} f_bs={f_bs} R2={metrics['r2']:.4f} NRMSE={metrics['nrmse']:.4f}")
if sweep_results:
    best = max(sweep_results, key=lambda entry: entry["metrics"]["r2"])
    print("\nBest configuration:")
    print(f"Q={best['num_qubits']} b={best['b']:.2f} warmup={best['warmup']} f_bs={best['f_bs']} n_res={best['n_reservoirs']} R2={best['metrics']['r2']:.4f} NRMSE={best['metrics']['nrmse']:.4f}")
else:
    print("No sweep results available.")


Training complete: 250 samples, 30 features
Q=4 b=-0.50 warmup=5 f_bs=[0.08, 0.1, 0.12] R2=-1.9734 NRMSE=1.7244
Training complete: 250 samples, 30 features
Q=4 b=-0.50 warmup=8 f_bs=[0.08, 0.1, 0.12] R2=-0.1563 NRMSE=1.0753
Training complete: 250 samples, 30 features
Q=4 b=-0.50 warmup=5 f_bs=[0.11, 0.1375, 0.12375] R2=-0.2853 NRMSE=1.1337
Training complete: 250 samples, 30 features
Q=4 b=-0.50 warmup=8 f_bs=[0.11, 0.1375, 0.12375] R2=0.0679 NRMSE=0.9655
Training complete: 250 samples, 30 features
Q=4 b=-0.50 warmup=5 f_bs=[0.14, 0.16, 0.18] R2=-3.1782 NRMSE=2.0441
Training complete: 250 samples, 30 features
Q=4 b=-0.50 warmup=8 f_bs=[0.14, 0.16, 0.18] R2=-0.9087 NRMSE=1.3815
Training complete: 250 samples, 40 features
Q=4 b=-0.50 warmup=5 f_bs=[0.09, 0.11, 0.13, 0.15] R2=-0.5875 NRMSE=1.2600
Training complete: 250 samples, 40 features
Q=4 b=-0.50 warmup=8 f_bs=[0.09, 0.11, 0.13, 0.15] R2=0.1101 NRMSE=0.9433
Training complete: 250 samples, 30 features
Q=4 b=-0.40 warmup=5 f_bs=[0.08, 0